In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [2]:
# import all excel datasets using pandas
# excel files for the first set of data
# tsangkaris
sum_table1 = pd.read_excel("Supplementary Table 8.xlsx")
CB_1 = pd.read_excel("Mouse_Brain_-_Cerebellum_7_weeks.xlsx")
CC_1 = pd.read_excel("Mouse_Brain_-_Cortex_7_weeks.xlsx")
HT_1 = pd.read_excel("mouse_brain_-_hipothalamus_7_weeks-01.xlsx")
HC_1 = pd.read_excel("mouse_brain_-_hipocampus_7_weeks-01.xlsx")
MD_1 = pd.read_excel("mouse_brain_-_medulla_7_weeks_old-01.xlsx")
MB_1 = pd.read_excel("Mouse_Brain_-_Mid_Brain_7_weeks.xlsx")
OB_1 = pd.read_excel("mouse_brain_-_olfactory_balb_7_weeks-01.xlsx")

# taraslia
sum_table2 = pd.read_excel("TARASLIA et al TABLE 1.xls", skiprows=1)
CB_2 = pd.read_excel("TARASLIA et al TABLE 6.xls", skiprows=1)
CC_2 = pd.read_excel("TARASLIA et al TABLE 8.xls", skiprows=1)
HT_2 = pd.read_excel("TARASLIA et al TABLE 7.xls", skiprows=1)
HC_2 = pd.read_excel("TARASLIA et al TABLE 3.xls", skiprows=1)
MD_2 = pd.read_excel("TARASLIA et al TABLE 5.xls", skiprows=1)
MB_2 = pd.read_excel("TARASLIA et al TABLE 4.xls", skiprows=1)
OB_2 = pd.read_excel("TARASLIA et al TABLE 2.xls", skiprows=1)

MB_2.rename(columns={'Protein MW  (Da)':'Protein MW (Da)'}, inplace=True)


First of all try to change the column names so that they are all the same for both origins.

In [3]:
# replace in series 1 tables the accession with entry name
mapping_df = sum_table1[['Entry', 'Entry name']].copy()

dfs_series1 = [CB_1, CC_1, HT_1, HC_1, MD_1, MB_1, OB_1]

for df in dfs_series1:
    # Use the map function to replace values in the 'Accession' column
    df['Accession'] = df['Accession'].map(mapping_df.set_index('Entry')['Entry name'])

# rename the 'entry name' in the supplementary table 1
sum_table1.rename(columns={'Entry name':'Accession name'}, inplace=True)
sum_table1.drop('Entry', axis=1, inplace=True)

sum_table2.rename(columns={'Accession Name':'Accession name'}, inplace=True)

In [4]:
# create lists of the dataframes from the two different sources
dfs_series1 = [CB_1, CC_1, HT_1, HC_1, MD_1, MB_1, OB_1]
dfs_series2 = [CB_2, CC_2, HT_2, HC_2, MD_2, MB_2, OB_2]

# convert Da to kDa in 2nd series of documents and rename the columns
for df in dfs_series2:
    df['Protein MW (Da)'] = df['Protein MW (Da)'] / 1000
    df.rename(columns={'Accession Name':'Accession name',
                       'Protein MW (Da)': 'Molecular weight (kDa)',
                       'pI-value':'pI-Value'}, inplace=True)

# renaming the column names of the first series
for df in dfs_series1:
    df.rename(columns={'Score': 'Mascot Score',
                       'Accession': 'Accession name',
                       'Coverage':'Coverage (%)',
                       '# Proteins': 'Proteins',
                       '# Unique Peptides': 'Unique Peptides',
                       '# Peptides':'Peptides',
                       '# PSMs':'PSMs',
                       '# AAs': 'Amino Acids',
                       'MW [kDa]': 'Molecular weight (kDa)',
                       'calc. pI': 'pI-Value'}, inplace=True)


In [5]:
sum_table1.head()

,Accession name,Protein names,Cerebral cortex,Olfactory Bulb,Hippocampus,Hypothalamus,Mid brain,Cerebellum,Medulla
0,A0A067XG52_MOUSE,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,A0A4W9_MOUSE,Neuronal growth regulator 1,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,SMKY_MOUSE,Sperm motility kinase Y (EC 2.7.11.1),0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,A0JLR7_MOUSE,Ahnak protein (Fragment),0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
sum_table2.head()

,Accession name,Protein name,Protein MW,pI-Value,Brain part,Subcellular localisation,Function
0,1433B_MOUSE,14-3-3 protein beta/alpha,28183,4.62,"OB, HT, MD",Cytoplasm,"Transcription rergulation, cytoplasmic sequest..."
1,1433E_MOUSE,14-3-3 protein epsilon,29326,4.50,"OB, MB, HT, HC, CB, MD",Cytoplasm,"Cerebral cortex development, hippocampus devel..."
2,1433F_MOUSE,14-3-3 protein eta,28365,4.66,OB,Cytoplasm,"Glucocorticoid catabolism, glucocorticoid rece..."
3,1433G_MOUSE,14-3-3 protein gamma,28456,4.70,"OB, CC, MB, HT, HC, CB, MD",Cytoplasm,"Neuronal differentiation, regulation of synapt..."
4,1433T_MOUSE,14-3-3 protein theta,28046,4.54,"OB, MD",Cytoplasm,"Trasnscription, signal transduction"


In [7]:
# check if all accession names of each brain region exist in the supplementary table

# create a list out of the accession names of the supplementary tables
prots1 = sum_table1['Accession name'].tolist()

for df in dfs_series1:
    # create a list out of the accession names of each df
    accession_list = df['Accession name'].tolist()

    # check that all proteins from each brain region exists in the supplementary table
    # if not return the index, and the data frame name
    for i, accession_name in enumerate(accession_list):
        if accession_name not in prots1:
            print(f"Accession name '{accession_name}' at index {i} in DataFrame {df.name} does not exist in the supplementary table.")

In [8]:
prots2 = sum_table2['Accession name'].tolist()

for df_index, df in enumerate(dfs_series2):
    # Check if 'Accession name' column exists in the DataFrame
    if 'Accession name' in df.columns:
        accession_list = df['Accession name'].tolist()

        # Check that all proteins from each brain region exist in the supplementary table
        # If not, return the index, and the data frame index
        for i, accession_name in enumerate(accession_list):
            if accession_name not in prots2:
                print(f"Accession name '{accession_name}' at index {i} in DataFrame {df_index} does not exist in the supplementary table.")
    else:
        print(f"DataFrame {df_index} does not have the 'Accession name' column.")


Accession name 'CV028_MOUSE' at index 5 in DataFrame 1 does not exist in the supplementary table.
Accession name 'CX4NB_MOUSE' at index 2 in DataFrame 5 does not exist in the supplementary table.
Accession name 'CA093_MOUSE' at index 5 in DataFrame 6 does not exist in the supplementary table.


In [9]:
# Add those accession names in the supplementary table

sum_table2 = sum_table2.append({'Accession name':'CV028_MOUSE', 'Brain part': 'CC'}, ignore_index=True)
sum_table2 = sum_table2.append({'Accession name':'CX4NB_MOUSE', 'Brain part': 'MB'}, ignore_index=True)
sum_table2 = sum_table2.append({'Accession name':'CA093_MOUSE', 'Brain part': 'OB'}, ignore_index=True)

/var/folders/4b/f4zzvdd97mdb6v7rmbgvl76w0000gn/T/ipykernel_8904/3656710357.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sum_table2 = sum_table2.append({'Accession name':'CV028_MOUSE', 'Brain part': 'CC'}, ignore_index=True)
/var/folders/4b/f4zzvdd97mdb6v7rmbgvl76w0000gn/T/ipykernel_8904/3656710357.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sum_table2 = sum_table2.append({'Accession name':'CX4NB_MOUSE', 'Brain part': 'MB'}, ignore_index=True)
/var/folders/4b/f4zzvdd97mdb6v7rmbgvl76w0000gn/T/ipykernel_8904/3656710357.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sum_table2 = sum_table2.append({'Accession name':'CA093_MOUSE', 'Brain part': 'OB'}, ignore_index=True)


Three accession names existed in the 6 regions but didn't exist in the Summarized table in the Taraslia data.

### Protein and Brain regions Collections
Try out the generation of unique protein accession_name collection AND keep in dataframe which are unique from which origin, and which are common as well. Also, create a collection for the unique brain regions.

In [35]:
# create a new dataframe that will contain which proteins are unique to which origin (series 1: tsangkaris, series 2: taraslia)
proteins = prots1 + prots2

proteins_all = pd.DataFrame({'Accession name':pd.Series(proteins).unique(), 'Origin': None})
prots1_series = pd.Series(prots1)
prots2_series = pd.Series(prots2)

# identify common, and unique proteins with respect to each origin
common_prots = prots1_series[prots1_series.isin(prots2_series)]
unique_prots1 = prots1_series[~prots1_series.isin(common_prots)]
unique_prots2 = prots2_series[~prots2_series.isin(common_prots)]

# loop through each row in the DataFrame
for index, row in proteins_all.iterrows():
    accession_name = row['Accession name']
    
    # Check the group to which the 'Accession name' belongs and update the 'Origin' column
    if accession_name in common_prots.values:
        proteins_all.at[index, 'Origin'] = ['Tsagkaris', 'Taraslia']
    elif accession_name in unique_prots1.values:
        proteins_all.at[index, 'Origin'] = 'Tsagkaris'
    elif accession_name in unique_prots2.values:
        proteins_all.at[index, 'Origin'] = 'Taraslia'


proteins_all.head()

,Accession name,Origin
0,A0A067XG52_MOUSE,Tsagkaris
1,AUTS2_MOUSE,Tsagkaris
2,A0A4W9_MOUSE,Tsagkaris
3,SMKY_MOUSE,Tsagkaris
4,A0JLR7_MOUSE,Tsagkaris
